In [53]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
import threading

In [54]:
coupon = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析/input/coupon.csv')
data   = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析/input/前處理完資料.csv')

In [55]:
coupon.head()

,Task_ID,Coupon_Title
0,305931,50元優惠券
1,305933,熟客優惠券
2,305937,推播優惠
3,305939,15分未達優惠
4,305941,15分未達優惠


In [56]:
data_merge = data.merge(coupon,left_on='id',right_on='Task_ID',how='left')

In [57]:
data_merge[['UserCreditValue','Coupon_Title']]

,UserCreditValue,Coupon_Title
0,50.0,50元優惠券
1,100.0,熟客優惠券
2,0.0,NaN
3,0.0,NaN
4,NaN,NaN
...,...,...
537986,100.0,NaN
537987,0.0,NaN
537988,0.0,NaN
537989,0.0,NaN


# 有29076筆資料，雖然沒有使用優惠券但UserCreditValue仍不為0

In [58]:
優惠券_無資料_但有折扣 = data_merge[(data_merge['UserCreditValue']!=0) & ~(np.isnan(data_merge['UserCreditValue'])) & (pd.isna(data_merge['Coupon_Title']))]

In [65]:
def 折扣資料補齊(UserCreditValue,Coupon_Title):
    if UserCreditValue !=0 and np.isnan(UserCreditValue) == False and pd.isna(Coupon_Title):
        return str(round(UserCreditValue,0)).replace('.0','') + '元折價券'
    else:
        pass

In [66]:
data_merge['Coupon_Title_補'] = data_merge.apply(lambda x:折扣資料補齊(x['UserCreditValue'],x['Coupon_Title']),axis=1)

In [67]:
data_merge

,id,member_id,driver_id,TaskState,type,call_type,TaskRideTS,createtime,UserLat,UserLon,...,TaskWaitTimeFee,TaskDistanceFee,is_pay,pay_type,is_first_use,Color,取消統整,Task_ID,Coupon_Title,Coupon_Title_補
0,305931,17757,2460,7,1,1,1577808029,2020-01-01 00:00:00,24.983,121.462,...,0.0,0.0,1,1,0,#8fb47e,未取消,305931.0,50元優惠券,None
1,305933,16984,2093,7,1,1,1577808204,2020-01-01 00:03:00,24.152,120.680,...,0.0,0.0,1,2,0,#8fb47e,未取消,305933.0,熟客優惠券,None
2,305934,45264,541,7,2,1,1577808300,2020-01-01 00:04:00,25.021,121.471,...,0.0,0.0,1,1,0,#8fb47e,未取消,NaN,NaN,None
3,305935,14432,1923,-1,2,1,1577808360,2020-01-01 00:05:00,25.058,121.540,...,NaN,NaN,0,1,0,#b45959,取消，駕駛,NaN,NaN,None
4,305936,28726,2393,7,1,1,1577808311,2020-01-01 00:05:00,25.149,121.726,...,0.0,950.0,1,1,0,#8fb47e,未取消,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537986,851969,189421,2700,7,1,1,1640966188,2021-12-31 23:56:00,24.122,120.631,...,0.0,0.0,1,1,1,#8fb47e,未取消,NaN,NaN,100元折價券
537987,851970,163324,2628,7,1,1,1640966282,2021-12-31 23:58:00,24.162,120.649,...,0.0,0.0,1,1,0,#8fb47e,未取消,NaN,NaN,None
537988,851971,165182,2470,7,1,1,1640966262,2021-12-31 23:58:00,25.082,121.578,...,0.0,0.0,1,2,0,#8fb47e,未取消,NaN,NaN,None
537989,851972,60234,1529,7,1,1,1640966343,2021-12-31 23:59:00,24.995,121.337,...,0.0,0.0,1,1,0,#8fb47e,未取消,NaN,NaN,None


In [72]:
for x in tqdm(range(len(data_merge))):
    if pd.isna(data_merge.iloc[x,-2]) and data_merge.iloc[x,-1] != None:
        data_merge.iloc[x,-2] = data_merge.iloc[x,-1]

100%|██████████| 537991/537991 [03:27<00:00, 2591.67it/s] 


In [75]:
data_merge = data_merge.drop(columns=['Coupon_Title_補'])

In [79]:
data_merge.to_csv('/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析/output/優惠券合併補齊資料.csv',encoding='utf-8-sig',index=False,header=True)

In [80]:
有優惠券_缺折扣金額 = data_merge[((data_merge['UserCreditValue']==0) | np.isnan(data_merge['UserCreditValue'])) & ~(pd.isna(data_merge['Coupon_Title']))]